# ライブラリのインポート

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import glob
import re
from janome.tokenizer import Tokenizer

pd.set_option("display.max_colwidth", 100)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)

%matplotlib notebook

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# 前準備
### 元のテキストデータは以下からダウンロードしてください。
https://www.rondhuit.com/download.html#ldcc

In [ ]:
tnz = Tokenizer()
pth = pathlib.Path('c:/temp/text')

l = []
for p in pth.glob('**/*.txt') :
    # 記事データ以外はスキップ
    if p.name in ['CHANGES.txt','README.txt','LICENSE.txt']:
        continue

    # 記事データを開き、janomeで形態素解析⇒1行1単語の形式でリストに保持
    with open(p,'r',encoding='utf-8-sig') as f :
        l.extend([[p.parent.name, p.name, t.surface, t.part_of_speech] for s in f for t in tnz.tokenize(s)])

# リストをデータフレームに変換
df = pd.DataFrame(np.array(l))

# 列名を付与
df.columns = ['記事分類','ファイル名','単語','品詞']

# データフレームをcsv出力
df.to_csv('c:/temp/livedoor_corpus.csv', index=False)

# 記事分類毎の頻出単語Top5

In [ ]:
# 前準備で1ファイルにまとめたテキストデータを読み込む
df = pd.read_csv('c:/temp/livedoor_corpus.csv')

# 品詞は名詞一般のみに絞り込み
df = df[df['品詞'].str.startswith('名詞,一般')].reset_index(drop=True)

# 記事分類毎の単語の出現頻度を集計
gdf = pd.crosstab([df['記事分類'],df['単語'],df['品詞']],
                  '件数',
                  aggfunc='count',
                  values=df['単語']
                 ).reset_index()

# 記事分類毎の単語の出現頻度の降順で順位付け
gdf['順位'] = gdf.groupby(['記事分類'])['件数'].rank('dense', ascending=False)
gdf.sort_values(['記事分類','順位'], inplace=True)

# 記事分類毎の頻出単語Top5だけに絞り込み
gdf = gdf[gdf['順位'] <= 5]

# 記事分類毎の頻出単語Top5の確認
for k in gdf['記事分類'].unique():
    display(gdf[gdf['記事分類']==k])


# 気になる単語の周辺文の確認

In [ ]:
# 気になった単語を設定
word = 'T'

df = pd.read_csv('c:/temp/livedoor_corpus.csv')

# 気になった単語の出現位置を取得
idxes = df[(df['単語'] == word)
          &(df['品詞'].str.startswith('名詞,一般'))].index.values.tolist()

# ウィンドウサイズ（気になった単語の前後何単語まで確認するかの設定）
ws = 20

# 気になる単語の周辺文を取得
l = []
for i, r in df.loc[idxes, :].iterrows():
    s = i - ws
    e = i + ws
    tmp = df.loc[s:e, :]
    tmp = tmp[tmp['ファイル名']==r['ファイル名']]
    lm = list(map(str, tmp['単語'].values.tolist()))
    ss = ''.join(lm)
    l.append([r['記事分類'],r['ファイル名'],r['単語'],ss])
rdf = pd.DataFrame(np.array(l))
rdf.columns = ['記事分類','ファイル名','単語','単語周辺文']

rdf.head(5)

# テキストデータ加工（空白、改行、URL除去）

In [ ]:
import pandas as pd
import numpy as np
import pathlib
import glob
import re

from janome.tokenizer import Tokenizer
tnz = Tokenizer()

pth = pathlib.Path('c:/temp/text')

l = []
for p in pth.glob('**/*.txt') :
    # 記事データ以外はスキップ
    if p.name in ['CHANGES.txt','README.txt','LICENSE.txt']:
        continue
    
    # 記事データを開き、janomeで形態素解析⇒1行1単語の形式でリストに保持
    with open(p,'r',encoding='utf-8-sig') as f :
        s = f.read()
        s = s.replace('　', '')
        s = s.replace(' ', '')
        s = s.replace('\n', '')
        s = re.sub(r'http://.*\+[0-9]{4}', '', s)
        # 空白、改行、URLを除去
        l.extend([[p.parent.name, p.name, t.surface, t.part_of_speech] for t in tnz.tokenize(s)])

# リストをデータフレームに変換
df = pd.DataFrame(np.array(l))

# 列名を付与
df.columns = ['記事分類','ファイル名','単語','品詞']

# データフレームをcsv出力
df.to_csv('c:/temp/livedoor_corpus.csv', index=False)

# 記事分類毎の頻出単語Top5

In [ ]:
# 前準備で1ファイルにまとめたテキストデータを読み込む
df = pd.read_csv('c:/temp/livedoor_corpus.csv')

# 品詞は名詞一般のみに絞り込み
df = df[df['品詞'].str.startswith('名詞,一般')].reset_index(drop=True)

# 記事分類毎の単語の出現頻度を集計
gdf = pd.crosstab([df['記事分類'],df['単語'],df['品詞']],
                  '件数',
                  aggfunc='count',
                  values=df['単語']
                 ).reset_index()

# 記事分類毎の単語の出現頻度の降順で順位付け
gdf['順位'] = gdf.groupby(['記事分類'])['件数'].rank('dense', ascending=False)
gdf.sort_values(['記事分類','順位'], inplace=True)

# 記事分類毎の頻出単語Top5だけに絞り込み
gdf = gdf[gdf['順位'] <= 5]

# 記事分類毎の頻出単語Top5の確認
for k in gdf['記事分類'].unique():
    display(gdf[gdf['記事分類']==k])


# テキストデータの主成分分析（2次元）

In [ ]:
# 上段のセルにて取得した記事分類毎の頻出単語Top5をリストとして保持
words = gdf['単語'].unique().tolist()

df = pd.read_csv('c:/temp/livedoor_corpus.csv')
df = df[df['品詞'].str.startswith('名詞,一般')].reset_index(drop=True)
df = df[df['単語'].isin(words)]

# ファイルと記事分類毎の頻出単語Top5のクロス集計表を取得
xdf = pd.crosstab([df['記事分類'],df['ファイル名']],df['単語']).reset_index()
# 後に因子負荷量のラベルとして出力するため、リストとして保持
cls = xdf.columns.values.tolist()[2:]

# 後のグラフ表示のため、記事分類毎に分類番号を付与
ul = xdf['記事分類'].unique()
def _fnc(x):
    return ul.tolist().index(x)
xdf['分類番号'] = xdf['記事分類'].apply(lambda x : _fnc(x))

# 主成分を求めるための前準備
data = xdf.values
labels = data[:,0]
d = data[:, 2:-1].astype(np.int64)
k = data[:, -1].astype(np.int64)

# データの標準化 ※ 標準偏差は不偏標準偏差で計算
X = (d - d.mean(axis=0)) / d.std(ddof=1,axis=0)

# 相関行列を求めます
XX = np.round(np.dot(X.T,X) / (len(X) - 1), 2)

# 相関行列の固有値、固有値ベクトルを求めます
w, V = np.linalg.eig(XX)

# 第1主成分を求める
z1 = np.dot(X,V[:,0])

# 第2主成分を求める
z2 = np.dot(X,V[:,1])

# グラフ用オブジェクトの生成
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)

# グリッド線を入れる
ax.grid()

# 描画するデータの境界
lim = [-10.0, 10.0]
ax.set_xlim(lim)
ax.set_ylim(lim)

# 左と下の軸線を真ん中に持っていく
ax.spines['bottom'].set_position(('axes', 0.5))
ax.spines['left'].set_position(('axes', 0.5))
# 右と上の軸線を消す
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# 軸の目盛の間隔を調整
ticks = np.arange(-10.0, 10.0, 1.0)
ax.set_xticks(ticks)
ax.set_yticks(ticks)

# 軸ラベルの追加、位置の調整
ax.set_xlabel('Z1', fontsize=16)
ax.set_ylabel('Z2', fontsize=16, rotation=0)
ax.xaxis.set_label_coords(1.02, 0.49)
ax.yaxis.set_label_coords(0.5, 1.02)

from matplotlib.colors import ListedColormap
colors = ['red','blue','gold','olive','green','dodgerblue','brown','black','grey']
cmap = ListedColormap(colors)
a = np.array(list(zip(z1,z2,k,labels)))
df = pd.DataFrame({'z1':pd.Series(z1, dtype='float'),
                   'z2':pd.Series(z2, dtype='float'),
                   'k':pd.Series(k, dtype='int'),
                   'labels':pd.Series(labels, dtype='str'),
                  })

# 記事分類毎に色を変えてプロット
for l in df['labels'].unique():
    d = df[df['labels']==l]
    ax.scatter(d['z1'],d['z2'],c=cmap(d['k']),label=l)
    ax.legend()

# 描画
plt.show()


# 因子負荷量（2次元）

In [ ]:
# 最大の固有値に対応する固有ベクトルを横軸、最大から2番目の固有値に対応する固有ベクトルを縦軸とした座標。
V_ = np.array([(V[:,0]),V[:,1]]).T
V_ = np.round(V_,2)

# グラフ描画用のデータ
z1 = V_[:,0]
z2 = V_[:,1]

# グラフ用オブジェクトの生成
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111)

# グリッド線を入れる
ax.grid()

# 描画するデータの境界
lim = [-0.4, 0.4]
ax.set_xlim(lim)
ax.set_ylim(lim)

# 左と下の軸線を真ん中に持っていく
ax.spines['bottom'].set_position(('axes', 0.5))
ax.spines['left'].set_position(('axes', 0.5))
# 右と上の軸線を消す
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# 軸の目盛の間隔を調整
ticks = np.arange(-0.4, 0.4, 0.05)
ax.set_xticks(ticks)
ax.set_yticks(ticks)

# 軸ラベルの追加、位置の調整
ax.set_xlabel('Z1', fontsize=16)
ax.set_ylabel('Z2', fontsize=16, rotation=0)
ax.xaxis.set_label_coords(1.02, 0.49)
ax.yaxis.set_label_coords(0.5, 1.02)

# データのプロット
for (i,j,k) in zip(z1,z2,cls):
    ax.plot(i,j,'o')
    ax.annotate(k, xy=(i, j),fontsize=10)
    
# 描画
plt.show()


# テキストデータの主成分分析（3次元）

In [ ]:
# 上段のセルにて取得した記事分類毎の頻出単語Top5をリストとして保持
words = gdf['単語'].unique().tolist()

df = pd.read_csv('c:/temp/livedoor_corpus.csv')
df = df[df['品詞'].str.startswith('名詞,一般')].reset_index(drop=True)
df = df[df['単語'].isin(words)]

# ファイルと記事分類毎の頻出単語Top5のクロス集計表を取得
xdf = pd.crosstab([df['記事分類'],df['ファイル名']],df['単語']).reset_index()
# 後に因子負荷量のラベルとして出力するため、リストとして保持
cls = xdf.columns.values.tolist()[2:]

# 後のグラフ表示のため、記事分類毎に分類番号を付与
ul = xdf['記事分類'].unique()
def _fnc(x):
    return ul.tolist().index(x)
xdf['分類番号'] = xdf['記事分類'].apply(lambda x : _fnc(x))

# 主成分を求めるための前準備
data = xdf.values
labels = data[:,0]
d = data[:, 2:-1].astype(np.int64)
k = data[:, -1].astype(np.int64)

# データの標準化 ※ 標準偏差は不偏標準偏差で計算
X = (d - d.mean(axis=0)) / d.std(ddof=1,axis=0)

# 相関行列を求めます
XX = np.round(np.dot(X.T,X) / (len(X) - 1), 2)

# 相関行列の固有値、固有値ベクトルを求めます
w, V = np.linalg.eig(XX)

# 第1主成分を求める
z1 = np.dot(X,V[:,0])

# 第2主成分を求める
z2 = np.dot(X,V[:,1])

# 第3主成分を求める
z3 = np.dot(X,V[:,2])

# グラフ用オブジェクトの生成
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# グリッド線を入れる
ax.grid()

# 描画するデータの境界
lim = [-10.0, 10.0]
ax.set_xlim(lim)
ax.set_ylim(lim)

# 左と下の軸線を真ん中に持っていく
ax.spines['bottom'].set_position(('axes', 0.5))
ax.spines['left'].set_position(('axes', 0.5))
# 右と上の軸線を消す
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# 軸の目盛の間隔を調整
ticks = np.arange(-10.0, 10.0, 1.0)
ax.set_xticks(ticks)
ax.set_yticks(ticks)

# 軸ラベルの追加、位置の調整
ax.set_xlabel('Z1', fontsize=16)
ax.set_ylabel('Z2', fontsize=16, rotation=0)
ax.xaxis.set_label_coords(1.02, 0.49)
ax.yaxis.set_label_coords(0.5, 1.02)

from matplotlib.colors import ListedColormap
colors = ['red','blue','gold','olive','green','dodgerblue','brown','black','grey']
cmap = ListedColormap(colors)

a = np.array(list(zip(z1,z2,z3,k,labels)))
df = pd.DataFrame({'z1':pd.Series(z1, dtype='float'),
                   'z2':pd.Series(z2, dtype='float'),
                   'z3':pd.Series(z3, dtype='float'),
                   'k':pd.Series(k, dtype='int'),
                   'labels':pd.Series(labels, dtype='str'),
                  })

for l in df['labels'].unique():
    d = df[df['labels']==l]
    ax.scatter(d['z1'],d['z2'],d['z3'],c=cmap(d['k']),label=l)
    ax.legend()

# 描画
plt.show()

# 因子負荷量（3次元）

In [ ]:
# 最大の固有値に対応する固有ベクトルを横軸、最大から2番目の固有値に対応する固有ベクトルを縦軸とした座標。
V_ = np.array([(V[:,0]),V[:,1],V[:,2]]).T
V_ = np.round(V_,2)

# グラフ描画用のデータ
z1 = V_[:,0]
z2 = V_[:,1]
z3 = V_[:,2]

# グラフ用オブジェクトの生成
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

# グリッド線を入れる
ax.grid()

# 描画するデータの境界
lim = [-0.4, 0.4]
ax.set_xlim(lim)
ax.set_ylim(lim)

# 左と下の軸線を真ん中に持っていく
ax.spines['bottom'].set_position(('axes', 0.5))
ax.spines['left'].set_position(('axes', 0.5))
# 右と上の軸線を消す
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

# 軸の目盛の間隔を調整
ticks = np.arange(-0.4, 0.4, 0.05)
ax.set_xticks(ticks)
ax.set_yticks(ticks)

# 軸ラベルの追加、位置の調整
ax.set_xlabel('Z1', fontsize=16)
ax.set_ylabel('Z2', fontsize=16)
ax.set_zlabel('Z3', fontsize=16)

ax.xaxis.set_label_coords(1.02, 0.49)
ax.yaxis.set_label_coords(0.5, 1.02)

# データのプロット
for zdir, x, y, z in zip(cls, z1, z2, z3):
    ax.scatter(x, y, z)
    ax.text(x, y, z, zdir)

# 描画
plt.show()
